In [ ]:
## Q3. "Plausibly Exogenous"

In [ ]:
### 1. How can instrumental variables Z help identify Beta in the cannonical Wright 1934 model? 

In [ ]:
Say we have identified an instrument Z that satisfies "E(u'Z) = 0." 

In [ ]:
 Modify the framework we developed for exploring the Wright (1934) model so that the data generating process 
 allows for = 0, and explore how estimator b(gamma) varies with gamma.

In [ ]:
### Data Generating Process Allowing for gamma different from 0

In [16]:
from scipy.stats import distributions as iid

def linear_dgp(N,beta,gamma,pi,sigma_u,sigma_v):
    u = iid.norm(scale=sigma_u).rvs(N)
    v = iid.norm(scale=sigma_v).rvs(N)
    Z = iid.norm().rvs(N)

    X = Z*pi + v
    y = X*beta + Z*gamma + u

    df = pd.DataFrame({'y':y,'x':X,'z':Z,'Constant':1})

    return df[['y']],df[['Constant','x']],df[['Constant','z']]

In [22]:
import numpy as np

# Structural parameters;
(alpha,beta, gamma) = (-1,2, 0.05)     
sigma = {'u':1/2,'v':1/3}
mu = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=mu['u'], scale=sigma['u'])  # Demand shocks
v = iid.norm(loc=mu['v'], scale=sigma['v'])  # Supply shocks

# Reduced form coefficients
pi = [[-beta/(alpha - beta), -1/(alpha - beta)],
     [ alpha/(alpha - beta), 1/(alpha - beta)]]

def draw_b(N,beta,gamma,pi,sigma_u,sigma_v):
    """
    Generate a random variate $b$ from a sample of $N$ draws from a function linear_dgp.
    """
    y,X,Z =  linear_dgp(N,beta,gamma,pi,sigma_u,sigma_v)

    return np.linalg.solve(Z.T@X,Z.T@y) # Solve normal eqs

b = draw_b(10000,beta, gamma, pi, sigma['u'], sigma['v'])

print(b)

ValueError: operands could not be broadcast together with shapes (10000,) (2,2) 

TypeError: linear_dgp() missing 5 required positional arguments: 'beta', 'gamma', 'pi', 'sigma_u', and 'sigma_v'